# Application: Build a qualitative researcher & launch an interview
This notebook uses EDSL to simulate qualitative research interviews. In a series of steps we construct an AI agent representing a qualitative researcher and launch an interview with specified topics and interview subjects.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/qualitative_research.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl import QuestionFreeText, Agent, Scenario, Model
m = Model("gpt-4-1106-preview")

import textwrap
from rich import print

In [3]:
def construct_subject(name, traits = {}):
    return Agent(name = name, traits = traits)
 
def construct_researcher(product, topics):
    # Note we use an f-string to populate the background. 
    # This should probably be moved to the agent's instructions as a system prompt.
    # Need to document Prompts! 
    background = textwrap.dedent(f"""\
    You are an expert qualitative researcher. 
    You are researching this product: {product}. 
    You are interested in conducting interviews to learn people's views on the following topics
    relating to the product: {topics}. 
    You know to ask questions that are appropriate to the age and experience of an interview subject.
    You know to not ask questions that an interview subject would not be able to answer, 
    e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
    You ask excellent follow-up questions.    
    """)
    return Agent(traits = {'background': background})

def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario({'subject': str(subject.traits), 'dialog_so_far': dialog_so_far})
    meta_q = QuestionFreeText(
        question_name = "next_question",
        question_text = """
        These are the biographic details of the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """
    )
    question_text = meta_q.by(m).by(researcher).by(scenario).run().select("next_question").first()
    return question_text

def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name = "question",
        question_text = f"""
        This is your current dialog with the interview subject: {dialog_so_far}. 
        You are now being asked:""" + question_text 
    )
    response = q_to_subject.by(m).by(subject).run().select("question").first() 
    return response

def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)
    return {"question": question_text, "response": response}

def dialog_to_string(d):
    return "\n".join([f"Question: {d['question']}\nResponse: {d['response']}" for d in d])

In [4]:
subject = construct_subject(name = "Alice", traits = {'gender': "female", 'age': 12})
researcher = construct_researcher(product = "cassette tapes", topics = "technology, music")
result = ask_question(subject, researcher, "")
print(result)

{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it? Do you use any 
apps or devices?',
    'response': "I really like listening to pop music and some K-pop too! It's super catchy and the dances are 
really cool. I usually listen to my music on my phone with headphones. My friends and I share playlists on Spotify,
and sometimes I use YouTube to watch music videos, especially when I want to learn the dance moves!"
}

In [5]:
dialog_so_far = []
for i in range(5):
    result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
    print(result)
    dialog_so_far.append(result)
print(dialog_so_far)

{
    'question': 'What kind of music do you like to listen to, and how do you usually listen to it? Do you use any 
apps or devices?',
    'response': "I really like listening to pop music and some K-pop too! It's super catchy and the dances are 
really cool. I usually listen to my music on my phone with headphones. My friends and I share playlists on Spotify,
and sometimes I use YouTube to watch music videos, especially when I want to learn the dance moves!"
}

{
    'question': "That's great! It sounds like you really enjoy the energy and visuals that come with pop and K-pop.
Have you ever had the experience of listening to music on older formats like CD players or cassette tapes? If so, 
what did you think about that experience compared to using your phone and apps like Spotify and YouTube?",
    'response': "Oh, I've never really used CD players or cassette tapes myself, but my parents have some old CDs 
and a tape player in the attic. They've shown me how they work once, and it was kind of cool, like a music time 
machine! But it's a lot more work to find the song you want, and you can’t make playlists as easily as on Spotify. 
I think using my phone is way more convenient because I can carry all my favorite songs with me and I don't have to
carry around a bunch of CDs or tapes. Plus, on YouTube, I can see the music videos, and I don't think you can do 
that with CDs and tapes."
}

{
    'question': "It's fascinating to hear your perspective on the convenience of digital music platforms. Since you
mentioned the idea of a 'music time machine,' I'm curious, do you feel like something is lost or gained in the way 
we experience music now, compared to the older formats like cassette tapes and CDs that your parents used? For 
example, the artwork, the physical collection, or maybe the anticipation of waiting for a song to come on?",
    'response': "Yeah, I think there's something kind of special about the older ways of listening to music. Like, 
with CDs and tapes, you get to see the artwork on the cover and the little booklet inside that sometimes has the 
lyrics or pictures of the band. It's like having a little piece of the artist that you can hold onto. And I guess 
there's also that excitement when you're waiting for your favorite song to play on a tape or the radio, which you 
don't really get when you can just click and play any song instantly. But even though that stuff is cool, I still 
prefer being able to listen to music easily on my phone. It's just so much more convenient, and I can still see 
artwork and stuff online if I want to."
}

{
    'question': "You've given some thoughtful insights about the tangible aspects of older music formats. Given 
your appreciation for the artwork and the physical aspect of CDs and tapes, do you think you would enjoy collecting
physical copies of music from your favorite artists, similar to how some people collect vinyl records as a hobby? 
Why or why not?",
    'response': "I think it would be really cool to collect physical copies of music from my favorite artists! It's
like having a piece of them and their art. Even though I love the convenience of digital music, having a special 
edition CD or a cool tape would be awesome because it's unique and has the artwork and maybe even some notes from 
the artists. It would be fun to have a collection to show to my friends and pass down later, kind of like a memory 
box of my favorite music. But I'd probably still listen to the music on my phone most of the time because it's 
easier. So, yeah, I would enjoy collecting them, but more as a hobby and not the main way I listen to music."
}

{
    'question': "It's wonderful to hear that you see the value in both the convenience of digital music and the 
charm of physical collections. Considering your interest in possibly collecting CDs or tapes, how do you think your
music listening experience might change if you had a collection of physical copies? Do you think you would engage 
with the music differently, perhaps by taking the time to sit and listen to an album all the way through, or by 
reading the liner notes and lyrics?",
    'response': "If I had a collection of physical copies, I think my music listening experience would be a bit 
more intentional. I'd probably take the time to sit and really listen to an album from start to finish, which I 
don't do as much on my phone because I skip around a lot. Reading the liner notes and lyrics would be super cool 
too. It's like getting a behind-the-scenes look at what the artists were thinking. It would make me feel more 
connected to the music and the artists I love. Plus, it would be a fun way to spend time when I'm not on my phone 
or doing other activities."
}

[
    {
        'question': 'What kind of music do you like to listen to, and how do you usually listen to it? Do you use 
any apps or devices?',
        'response': "I really like listening to pop music and some K-pop too! It's super catchy and the dances are 
really cool. I usually listen to my music on my phone with headphones. My friends and I share playlists on Spotify,
and sometimes I use YouTube to watch music videos, especially when I want to learn the dance moves!"
    },
    {
        'question': "That's great! It sounds like you really enjoy the energy and visuals that come with pop and 
K-pop. Have you ever had the experience of listening to music on older formats like CD players or cassette tapes? 
If so, what did you think about that experience compared to using your phone and apps like Spotify and YouTube?",
        'response': "Oh, I've never really used CD players or cassette tapes myself, but my parents have some old 
CDs and a tape player in the attic. They've shown me how they work once, and it was kind of cool, like a music time
machine! But it's a lot more work to find the song you want, and you can’t make playlists as easily as on Spotify. 
I think using my phone is way more convenient because I can carry all my favorite songs with me and I don't have to
carry around a bunch of CDs or tapes. Plus, on YouTube, I can see the music videos, and I don't think you can do 
that with CDs and tapes."
    },
    {
        'question': "It's fascinating to hear your perspective on the convenience of digital music platforms. Since
you mentioned the idea of a 'music time machine,' I'm curious, do you feel like something is lost or gained in the 
way we experience music now, compared to the older formats like cassette tapes and CDs that your parents used? For 
example, the artwork, the physical collection, or maybe the anticipation of waiting for a song to come on?",
        'response': "Yeah, I think there's something kind of special about the older ways of listening to music. 
Like, with CDs and tapes, you get to see the artwork on the cover and the little booklet inside that sometimes has 
the lyrics or pictures of the band. It's like having a little piece of the artist that you can hold onto. And I 
guess there's also that excitement when you're waiting for your favorite song to play on a tape or the radio, which
you don't really get when you can just click and play any song instantly. But even though that stuff is cool, I 
still prefer being able to listen to music easily on my phone. It's just so much more convenient, and I can still 
see artwork and stuff online if I want to."
    },
    {
        'question': "You've given some thoughtful insights about the tangible aspects of older music formats. Given
your appreciation for the artwork and the physical aspect of CDs and tapes, do you think you would enjoy collecting
physical copies of music from your favorite artists, similar to how some people collect vinyl records as a hobby? 
Why or why not?",
        'response': "I think it would be really cool to collect physical copies of music from my favorite artists! 
It's like having a piece of them and their art. Even though I love the convenience of digital music, having a 
special edition CD or a cool tape would be awesome because it's unique and has the artwork and maybe even some 
notes from the artists. It would be fun to have a collection to show to my friends and pass down later, kind of 
like a memory box of my favorite music. But I'd probably still listen to the music on my phone most of the time 
because it's easier. So, yeah, I would enjoy collecting them, but more as a hobby and not the main way I listen to 
music."
    },
    {
        'question': "It's wonderful to hear that you see the value in both the convenience of digital music and the
charm of physical collections. Considering your interest in possibly collecting CDs or tapes, how do you think your
music listening experience might change if you ha

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>